1. 사람 + 총 + 칼 인식    
2. Arcface를 모든 사람에 대해 적용(Me, NotME)  
3. NotMe이면서 사람과 총 혹은 칼의 bounding box가 겹치는 경우에는 위험인으로 분류 -> 한 번 위험인으로 분류된 사람의 ID는 dangerous_ids로 관리됨 -> dangerous_ids 중 하나에 해당하는 사람은 위에 "Dangerous person"이라고 뜸
4. pose estimation을 dangerous person에 대해 적용(왼팔들기, 오른팔들기, 양팔들기)  
5. 출력화면 사이즈를 키움  
6. 버그수정: 초기에 무기와 교차하여 dangerous_ids에 들어갔더라도, ArcFace로 내 얼굴임이 확인되면(=나 자신이 무기를 소지한 상황) 그 사람을 위험 인물에서 제외  
7. Kobukki robot이 dangerous_ids 내에 있는 사람을 쫓아가도록 하면 될 듯  

+ Yolo segmentation까지 추가
+ 현재 YOLO를 사용하고 있으므로, YOLOv8-seg 모델을 사용하는 것이 가장 간단함.

총이나 칼을 들고 있는 손의 정확한 위치를 파악해서 정밀하게 무기를 감지할 수 있음
옷 색깔이나 패턴을 인식해서 사람 구별 정확도 높아짐
제스처 구분 정확도 높아짐 
YOLOv8-seg 모델 사용


- v2: seg 모델 학습되어 있는 거 가져와서 테스트 (이전에 학습시켜두었던 yolo 모델은 가져오지 않음). seg + arcface
- v3: seg + arcface + deepsort (사람)
  
- v4:
Segmentation(사람에 한정) + Detection(사람, 총, 칼)   + arcface + deepsort
detection은 이전에 학습시켰던 epoch180.pt 사용
detection먼저 하고 사람인 경우에만 seg
(사람 bounding box 안에서 segmentation 되는듯)

개선할 점: 얼굴인식은 한 번만 (어차피 카메라가 목표대상을 계속 따라가기 때문에)
- tracking id 당 한 번씩만

- v5: tracking id 당 한 번씩만 얼굴 인식

v4까지는 느려도 잘 되는데 v5부터는 처음부터 끊김

0213v4: 0205v5에서 arcface 개선
얼굴 인식 잘 안됨
v5: 얼굴인식 잘 됨


In [1]:
#1. 얼굴 학습시키는 부분
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

def initialize_arcface():
    app = FaceAnalysis(name="buffalo_l")  # ArcFace 모델 (buffalo_l은 기본 권장)
    app.prepare(ctx_id=-1, det_size=(640, 640))  # GPU: ctx_id=0, CPU: -1
    return app

def get_face_embedding(app, image_bgr):
    # ArcFace의 app.get()은 BGR 형식으로 이미지를 받기도 합니다.
    # 만약 RGB가 필요하면 cvtColor로 변환하세요.
    faces = app.get(image_bgr)
    if len(faces) > 0:
        return faces[0].embedding  # 첫 번째 얼굴의 임베딩
    else:
        return None

def generate_average_embedding(app, folder_path):
    embeddings = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"이미지 로드 실패: {img_path}")
                continue
            
            embedding = get_face_embedding(app, image)
            if embedding is not None:
                embeddings.append(embedding)
            else:
                print(f"얼굴 검출 실패: {img_path}")
    
    if len(embeddings) == 0:
        raise ValueError("임베딩을 하나도 생성하지 못했습니다.")
    
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

if __name__ == "__main__":
    app = initialize_arcface()
    # 내 얼굴 사진 폴더
    my_face_folder = "C:/myface"  
    my_face_embedding = generate_average_embedding(app, my_face_folder)
    np.save("my_face_embedding.npy", my_face_embedding)  # 필요 시 저장
    print("내 얼굴 평균 임베딩 생성 완료.")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


내 얼굴 평균 임베딩 생성 완료.


In [5]:
import cv2
import numpy as np
import time
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
import mediapipe as mp

############################
# 1) 모델 및 함수 초기화
############################
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# YOLO Detection 모델 로드
detection_model = YOLO("C:/epoch180.pt").to(device)

# YOLO Segmentation 모델 로드
segmentation_model = YOLO("yolov8n-seg.pt").to(device)

# YOLO Face Detection 모델 로드
face_model = YOLO("C:/best.pt").to(device)

# DeepSORT 초기화
tracker = DeepSort(max_age=30, n_init=3, embedder='mobilenet', half=True, embedder_gpu=True)

# ArcFace 초기화
arc_app = FaceAnalysis(name="buffalo_l")
arc_app.prepare(ctx_id=-1, det_size=(640, 640))  # ctx_id=-1: CPU, ctx_id=0: GPU
my_face_embedding = np.load("my_face_embedding.npy")  # 내 얼굴의 평균 임베딩

# MediaPipe Pose Estimation 초기화
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# ArcFace 속도 개선: 초기 10 프레임 동안 연속 실행
INITIAL_FRAME_LIMIT = 10
frame_count = 0
arcface_enabled = True
arcface_result = None  # ArcFace 결과를 저장하여 연속적으로 유지


def get_face_embedding(face_image, arc_app):
    faces = arc_app.get(face_image)
    if len(faces) > 0:
        return faces[0].embedding
    return None

def is_my_face(face_embedding, my_face_embedding, threshold=0.4):
    if face_embedding is None:
        return False, 0.0
    similarity = cosine_similarity([face_embedding], [my_face_embedding])[0][0]
    return similarity > threshold, similarity

def segment_within_bboxes(image, bboxes):
    for (x1, y1, x2, y2) in bboxes:
        person_crop = image[y1:y2, x1:x2]
        if person_crop.size == 0:
            continue
        segmentation_results = segmentation_model(person_crop)
        if segmentation_results[0].masks is not None:
            for mask in segmentation_results[0].masks.data:
                mask = mask.cpu().numpy()
                mask_resized = cv2.resize(mask, (x2 - x1, y2 - y1))
                mask_binary = (mask_resized > 0.5).astype(np.uint8)
                color_mask = np.zeros_like(person_crop, dtype=np.uint8)
                color_mask[:, :, 1] = mask_binary * 255
                person_crop = cv2.addWeighted(person_crop, 1, color_mask, 0.5, 0)
                image[y1:y2, x1:x2] = person_crop
    return image

############################
# 2) 실시간 웹캡 캡처
############################
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

window_name = "YOLOv8 + DeepSORT + Segmentation + ArcFace + Pose"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 960, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    frame_count += 1
    if frame_count > INITIAL_FRAME_LIMIT:
        arcface_enabled = False  # 초기 10프레임 이후 ArcFace 비활성화
    
    detection_results = detection_model(frame)
    bboxes = []
    detections = []

    for box in detection_results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls)
        conf = float(box.conf)
        if class_id == 0:
            bboxes.append((x1, y1, x2, y2))
            detections.append(((x1, y1, x2 - x1, y2 - y1), conf, 0))

    frame = segment_within_bboxes(frame, bboxes)
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        track_id = track.track_id
        cv2.putText(frame, f"ID:{track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        if arcface_enabled:
            face_embedding = get_face_embedding(frame[y1:y2, x1:x2], arc_app)
            if face_embedding is not None:
                arcface_result = is_my_face(face_embedding, my_face_embedding, threshold=0.35)
        
        is_me, similarity = arcface_result if arcface_result is not None else (False, 0.0)
        text_color = (0, 255, 0) if is_me else (0, 0, 255)
        arcface_text = f"Me (sim={similarity:.2f})" if is_me else f"Not Me (sim={similarity:.2f})"
        cv2.putText(frame, arcface_text, (x1, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 480x640 1 personnnn, 1 gunnnnn, 100.8ms
Speed: 25.3ms preprocess, 100.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 1 person, 92.9ms
Speed: 2.8ms preprocess, 92.9ms inference, 8.1ms postprocess per image at shape (1, 3, 384, 640)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 480x640 1 personnnn, 1 gunnnnn, 107.4ms
Speed: 8.5ms preprocess, 107.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 352x640 1 person, 1 tv, 117.5ms
Speed: 2.7ms preprocess, 117.5ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)


...


0: 480x640 1 personnnn, 1 gunnnnn, 218.3ms
Speed: 9.5ms preprocess, 218.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 1 person, 1 cell phone, 227.2ms
Speed: 6.0ms preprocess, 227.2ms inference, 10.2ms postprocess per image at shape (1, 3, 448, 640)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 480x640 1 personnnn, 198.2ms
Speed: 19.6ms preprocess, 198.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 416x640 1 person, 167.4ms
Speed: 5.1ms preprocess, 167.4ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 480x640 1 personnnn, 182.4ms
Speed: 19.3ms preprocess, 182.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 416x640 1 person, 169.2ms
Speed: 9.3ms preprocess, 169.2ms inference, 5.2ms postprocess per image at shape (1, 3, 416, 640)

0: 480x640 1 personnnn, 232.8ms
Speed: 16.7ms preprocess, 232.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 416x640 1 person, 1 tv, 176.3ms
Speed: 5.4ms preprocess, 176.3ms inference, 10.2ms postprocess per image at shape (1, 3, 416, 640)

0: 480x640 1 personnnn, 149.5ms
Speed: 4.0ms preprocess, 149.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 416x640 1 person, 1 tv, 205.8ms
Speed: 5.4ms preprocess, 205.8ms inference, 10.1ms postprocess per image at shape (1, 3, 416, 640)

0: 480x640 1 personnnn, 149.2ms
Speed: 4.6ms preprocess, 149.2ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 416x640 1 person, 1 tv, 173.9ms
Speed: 3.7ms preprocess, 173.9ms inferen

In [ ]:
import torch

print("CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())